### Tomorrow.io Notes (free version)

Exploration of tomorrow.io's weather API

In [1]:
import requests
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from datetime import datetime, time, timedelta
import statsmodels.api as sm

KEY = "BSeLVQzecQssiPyaI4CBmkFdvMvGarYa" #add personal key here

#### 1 Overview

Tomorrow.io has two main data formats - data layers and insights. 
 - Data layers can be thought of as "raw data", ie. precipitation rate, temperature, soil moisture, etc.
 - Insights, such as probability of a flood or a fire, are calculated from the data layers.
 
The two data formats are retrieved in different ways - data layers are retrieved as timelines, while insights are retrieved as events. 

#### 2 Querying Locations

Locations can be queried as Points (free trial) or as LineStrings/Polygons (paid version), and should follow the [formats specified in the docs](https://docs.tomorrow.io/reference/api-formats#locations). You can also check out the [RFC 7946 appendix](https://datatracker.ietf.org/doc/html/rfc7946#appendix-A) for more specifics on how to format locations.

##### 2a Points

Points can be queried as `latitude,longitude` strings, like so:

In [ ]:
point = '30.2, 78.7'

##### 2b LineStrings

To query a LineString, use tomorrow.io's Location API to define a location first. You can read [Location API docs](https://docs.tomorrow.io/reference/post-locations) for more information, or use the function below:

**NOTE: need to make sure this works, can't do this with a free trial. but should work because it runs into a forbidden access error**

In [11]:
def get_line(coordinates,name):
    url = 'https://api.tomorrow.io/v4/locations?apikey=' + KEY
    payload = {
        "geometry": {
            "type": "LineString",
            "coordinates": coordinates},
        "name": name
    }
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json"
    }
    response = requests.request("POST", url, json=payload, headers=headers)
    return response

coordinates = [[30.3397,79.3156], [30.3641,79.3136]]
name = 'NH7'
get_line(coordinates,name)

<Response [403]>

##### 2c Polygons

Similarly, you can use the Location API to query a polygon. A sample function is written below:

**NOTE: not convinced that this works because it gives a 400 error instead of 403 (bad request)**

In [10]:
def get_polygon(coordinates,name):
    url = 'https://api.tomorrow.io/v4/locations?apikey=' + KEY
    payload = {
        "geometry": {
            "type": "Polygon",
            "coordinates": coordinates},
        "name": name
    }
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json"
    }
    response = requests.request("POST", url, json=payload, headers=headers)
    return response

coordinates = [[30.3397,79.3156], [30.3641,79.3136], [30.3,79.3]]
name = 'NH7'
get_polygon(coordinates,name)

<Response [400]>

#### 3 Retrieving Data Layers/Timelines

It would be best to first read the [Data Layers Overview](https://docs.tomorrow.io/reference/data-layers-overview) and the [Timelines Overview](https://docs.tomorrow.io/reference/timeline-overview).

Tomorrow.io requires some query params to retrieve a timeline, which are detailed on the [Get Timelines page](https://docs.tomorrow.io/reference/get-timelines). Some additional notes on each parameter:
 - `location:` input as described above
 - `fields:` full list of fields in alphabetical order found on the [Data Layers Overview](https://docs.tomorrow.io/reference/data-layers-overview). They are also categorized as [Core Layers](https://docs.tomorrow.io/reference/data-layers-core) and more specific layers - for example, [Air Quality](https://docs.tomorrow.io/reference/data-layers-air). 
    - When querying lines or polygons, you can append a suffix (`Max`, `Min`, `Avg`, `MaxTime/MinTime`). For example, entering `temperatureAvg` as a field will return the average temperature in the line/polygon. 
    - Check out the [Data Layers Overview](https://docs.tomorrow.io/reference/data-layers-overview) and the [Timelines Overview](https://docs.tomorrow.io/reference/timeline-overview) for limitations/specifics on suffixes. 
 - `units`, `timesteps`: full list of availability found on [Data Layers Overview](https://docs.tomorrow.io/reference/data-layers-overview).
 - `timezone`: function below sets default timezone as `America/New_York`, but can be changed by editing the function.
 - `startTime`, `endTime`: must be in ISO 8601 format - you can use the `datetime` module in Python to convert dates to this format if necessary.
    - Both are optional parameters - Tomorrow.io has built-in default values.

In [2]:
def call_point(point, fields, timestep, start=None, end=None):
    #perform API call
    url = 'https://api.tomorrow.io/v4/timelines?apikey=' + KEY
    querystring = {
        'location': point, 
        'fields': fields, 
        'units':'metric', 
        'timesteps': timestep, 
        'timezone': 'America/New_York',
        'startTime': start,
        'endTime': end
    }
    headers = {'Accept': 'application/json'}
    response = requests.request("GET", url, headers=headers, params=querystring)
    return response

In [2]:
def call_line(coordinates, fields, timestep):
    #perform API call
    url = 'https://api.tomorrow.io/v4/timelines?apikey=' + KEY
    
    payload = {
        'units':'metric', 
        'timesteps': timestep, 
        'location': point, 
        'fields': fields, 
        'timezone': 'America/New_York',
        #'startTime': start,
        #'endTime': end,
        'location': {
            'type': 'LineString',
            'coordinates': coordinates},
    }
    
    headers = {
    "Accept": "application/json",
    "Content-Type": "application/json"
    }
    
    response = requests.request("POST", url, json=payload, headers=headers)
    return response

In [3]:
point = '30.2, 78.5'
coordinates = [[79.3156,30.3397], [79.3136,30.3641,], [79.3,30.3]]
fields = ['precipitationIntensityMax', 'rainAccumulation', 'temperature', 'windSpeed', 'soilMoistureVolumetric0To10']
timestep = ['1h']

#data = call_point(point, fields, timestep)
data_line = call_line(coordinates, fields, timestep)

In [4]:
data_line.text

'{"data":{"timelines":[{"timestep":"1h","startTime":"2021-09-02T20:00:00-04:00","endTime":"2021-09-07T08:00:00-04:00","intervals":[{"startTime":"2021-09-02T20:00:00-04:00","values":{"rainAccumulation":0,"precipitationIntensityMax":0,"temperature":17.19,"windSpeed":0.69,"soilMoistureVolumetric0To10":33.73}},{"startTime":"2021-09-02T21:00:00-04:00","values":{"rainAccumulation":0,"precipitationIntensityMax":0,"temperature":20.89,"windSpeed":0.95,"soilMoistureVolumetric0To10":33.73}},{"startTime":"2021-09-02T22:00:00-04:00","values":{"rainAccumulation":0,"precipitationIntensityMax":0,"temperature":20.85,"windSpeed":0.78,"soilMoistureVolumetric0To10":33.71}},{"startTime":"2021-09-02T23:00:00-04:00","values":{"rainAccumulation":0,"precipitationIntensityMax":0,"temperature":21.34,"windSpeed":0.52,"soilMoistureVolumetric0To10":33.61}},{"startTime":"2021-09-03T00:00:00-04:00","values":{"rainAccumulation":0,"precipitationIntensityMax":0,"temperature":22.42,"windSpeed":0.25,"soilMoistureVolumetri

Tomorrow.io returns the data in a json-format, which needs some formatting to be readable in pandas (for further analysis):

In [29]:
def format_response(response):
    #text = json.loads(response.text)
    timelines = response['data']['timelines']
    nest = timelines[0]['intervals'] #list - each element is a dictionary with startTime and values
    startTimes = pd.DataFrame(nest)['startTime']

    val = nest[0]['values'] #first time-stamp value
    values = pd.DataFrame(val, index=[0])

    #iterate through all time-stamps
    for i in range(1, len(startTimes)):
        val = nest[i]['values']
        df = pd.DataFrame(val, index=[i])
        values = pd.concat([values, df], axis=0)

    data = pd.concat([startTimes, values], axis=1)
    return data

In [ ]:
json.loads(data_line.text)